In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!ls

attacks.py		     basetrainer.py    models
Audio Reconstruction.ipynb   CustomDataset.py  __pycache__
audio_training2.ipynb	     dataloader.py     saved
audio_training.ipynb	     evaluation.ipynb  spectogram.py
audio_training_output.ipynb  logs	       utils.py


## Extract data from zip file




In [4]:
# import package to unzip .7zip file
!sudo apt-get install -y p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-3+deb9u1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [5]:
# locally unzip the audio files in train.7z
# !cd ../Data/ && p7zip -d train.7z

In [6]:
# !ls ../Data/train/audio

## Imports

In [3]:
%matplotlib inline

import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage

import torch
from torch.utils.data import DataLoader

from utils import log_textfile
from basetrainer import BaseTrainer
from dataloader import load_data
from CustomDataset import CustomDatasetSimple
from models.resnet import ResNet, resnet34
from attacks import fgsm, pgd_linf

## Data Loader

In [4]:
DATADIR = '../Data/'

trainset, valset = load_data(DATADIR)

There are 21105 train and 2577 val samples


### Data subset

In [5]:
# from random import sample
# from random import shuffle

# # training set subsample
# trainset_sub = sample(trainset, 2000)
# len(trainset_sub)

# # validation subample
# shuffle(valset)
# valset_sub = valset[:1000]

# # subsample the training set from the validation for now!
# testset_sub = valset[1000:2000]
# print('length of trainset: ' + str(len(trainset_sub)) + ', length of validation set: ' +  str(len(valset_sub)), 
#       ', length of test set: ' + str(len(testset_sub)))

### Get filenames

In [6]:
trainset_dl = trainset
valset_dl = valset
#test_dl = testset_sub

train_filepaths = [i[2] for i in trainset_dl]
train_labels = [i[0] for i in trainset_dl]
valid_filepaths = [i[2] for i in valset_dl]
val_labels = [i[0] for i in valset_dl]
#test_filepaths = [i[2] for i in test_dl]
#test_labels = [i[0] for i in valset_dl]

In [13]:
# if False:
#     trainset_dl = trainset
#     valset_dl = valset
#     test_dl = testset_sub

#     # train
#     train_filepaths = [i[2] for i in trainset_dl]
#     train_audio_files = [wavfile.read(i)[1] for i in train_filepaths]
#     train_labels = [i[0] for i in trainset_dl]

#     train_spec = []
#     for i in range(len(train_audio_files)):
#       spec = pretty_spectrogram(train_audio_files[i].astype('float64'), fft_size = fft_size, step_size = step_size, log = True, thresh = spec_thresh)
#       height = spec.shape[0]
#       if height!=112:
#         spec = pad_dimesions(spec)
#       train_spec.append(spec)

#     # validation
#     valid_filepaths = [i[2] for i in valset_dl]
#     valid_audio_files = [wavfile.read(i)[1] for i in valid_filepaths]
#     val_labels = [i[0] for i in valset_dl]

#     valid_spec = []
#     for i in range(len(valid_audio_files)):
#       spec = pretty_spectrogram(valid_audio_files[i].astype('float64'), fft_size = fft_size, step_size = step_size, log = True, thresh = spec_thresh)
#       if height!=112:
#         spec = pad_dimesions(spec)
#       valid_spec.append(spec)

#     # test
#     test_filepaths = [i[2] for i in test_dl]
#     test_audio_files = [wavfile.read(i)[1] for i in test_filepaths]
#     test_labels = [i[0] for i in valset_dl]

#     test_spec = []
#     for i in range(len(test_audio_files)):
#       spec = pretty_spectrogram(test_audio_files[i].astype('float64'), fft_size = fft_size, step_size = step_size, log = True, thresh = spec_thresh)
#       if height!=112:
#         spec = pad_dimesions(spec)
#       test_spec.append(spec)

In [7]:
# from utils import get_mean_std
#mean, std = get_mean_std(train_data_loader)

mean=-3.1259581955996425
std=0.8961027914827521
batch_size=32
num_workers=8
data_train_sub = CustomDatasetSimple(train_filepaths, train_labels, mean, std)
data_valid_sub = CustomDatasetSimple(valid_filepaths, val_labels, mean, std)

train_data_loader = torch.utils.data.DataLoader(
    dataset=data_train_sub, batch_size=batch_size, shuffle=True,
    num_workers=num_workers)

valid_data_loader = torch.utils.data.DataLoader(
    dataset=data_valid_sub, batch_size=batch_size, shuffle=True,
    num_workers=num_workers)


In [ ]:
MODELNAME = 'full_dataset_normal'
LOGFILE_PATH = 'logs/' + MODELNAME

model = resnet34(pretrained=False, progress=False).cuda()

criterion = torch.nn.CrossEntropyLoss()

for param in model.parameters():
  param.requires_grad = True

trainer_pgd = BaseTrainer(model=model, 
                      train_dl=train_data_loader, 
                      valid_dl=valid_data_loader, 
                      criterion=criterion, 
                      model_filename=MODELNAME, 
                      n_epochs=3)

trainer_pgd.fit_model_new(optimizer=optim.Adam(model.parameters(), lr=.001), 
                      n_epochs=1, 
                      LOGFILE_PATH=LOGFILE_PATH,
                      model_filename=MODELNAME, 
                      attack=None, 
                      epsilon=0.3, 
                      alpha=alpha, 
                      num_iter=7)

In [ ]:
MODELNAME = 'full_dataset_fgsm03'
LOGFILE_PATH = 'logs/' + MODELNAME

model = resnet34(pretrained=False, progress=False).cuda()

criterion = torch.nn.CrossEntropyLoss()

for param in model.parameters():
  param.requires_grad = True

trainer_pgd = BaseTrainer(model=model, 
                      train_dl=train_data_loader, 
                      valid_dl=valid_data_loader, 
                      criterion=criterion, 
                      model_filename=MODELNAME, 
                      n_epochs=3)

trainer_pgd.fit_model_new(optimizer=optim.Adam(model.parameters(), lr=.001), 
                      n_epochs=1, 
                      LOGFILE_PATH=LOGFILE_PATH,
                      model_filename=MODELNAME, 
                      attack=None, 
                      epsilon=0.3, 
                      alpha=None, 
                      num_iter=None)

In [ ]:
MODELNAME = 'full_dataset_fgsm20'
LOGFILE_PATH = 'logs/' + MODELNAME

model = resnet34(pretrained=False, progress=False).cuda()

criterion = torch.nn.CrossEntropyLoss()

for param in model.parameters():
  param.requires_grad = True

trainer_pgd = BaseTrainer(model=model, 
                      train_dl=train_data_loader, 
                      valid_dl=valid_data_loader, 
                      criterion=criterion, 
                      model_filename=MODELNAME, 
                      n_epochs=3)

trainer_pgd.fit_model_new(optimizer=optim.Adam(model.parameters(), lr=.001), 
                      n_epochs=1, 
                      LOGFILE_PATH=LOGFILE_PATH,
                      model_filename=MODELNAME, 
                      attack=None, 
                      epsilon=2.0, 
                      alpha=None, 
                      num_iter=None)

In [ ]:
MODELNAME = 'full_dataset_pdg03'
LOGFILE_PATH = 'logs/' + MODELNAME

model = resnet34(pretrained=False, progress=False).cuda()

criterion = torch.nn.CrossEntropyLoss()

for param in model.parameters():
  param.requires_grad = True

trainer_pgd = BaseTrainer(model=model, 
                      train_dl=train_data_loader, 
                      valid_dl=valid_data_loader, 
                      criterion=criterion, 
                      model_filename=MODELNAME, 
                      n_epochs=3)

trainer_pgd.fit_model_new(optimizer=optim.Adam(model.parameters(), lr=.001), 
                      n_epochs=1, 
                      LOGFILE_PATH=LOGFILE_PATH,
                      model_filename=MODELNAME, 
                      attack=pgd_linf, 
                      epsilon=0.3, 
                      alpha=0.01, 
                      num_iter=7)

In [18]:
normalTrain1 = load_model('saved/normalTrain1')
full_dataset_fgsm03 = load_model('saved/full_dataset_fgsm03')
full_dataset_fgsm20 = load_model('saved/full_dataset_fgsm20')
full_dataset_pdg03 = load_model('saved/full_dataset_pdg03')
full_dataset_normal = load_model('saved/full_dataset_normal')

### No attacks

In [32]:
criterion = torch.nn.CrossEntropyLoss()

LOGFILE_PATH = 'logs/eval/full_dataset_normal'
trainer_full_dataset_normal = BaseTrainer(model=full_dataset_normal, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='eval_full_dataset_normal', 
                                   n_epochs=1)

trainer_full_dataset_normal.model.eval()
trainer_full_dataset_normal = trainer_full_dataset_normal.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=None)

20
[epoch: 1, batch:     21] loss: 0.00817 time model: 0.00666 acc: 0.92344
40
[epoch: 1, batch:     41] loss: 0.00740 time model: 0.00658 acc: 0.92344
60
[epoch: 1, batch:     61] loss: 0.00727 time model: 0.00656 acc: 0.92500
80
[epoch: 1, batch:     81] loss: 0.00714 time model: 0.00654 acc: 0.92500


In [34]:
LOGFILE_PATH = 'logs/eval/full_dataset_fgsm03'
trainer_full_dataset_fgsm03 = BaseTrainer(model=full_dataset_fgsm03, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='eval_full_dataset_fgsm03', 
                                   n_epochs=1)

trainer_full_dataset_fgsm03.model.eval()
trainer_full_dataset_fgsm03 = trainer_full_dataset_fgsm03.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=None)

20
[epoch: 1, batch:     21] loss: 0.01081 time model: 0.00661 acc: 0.89062
40
[epoch: 1, batch:     41] loss: 0.01098 time model: 0.00654 acc: 0.89219
60
[epoch: 1, batch:     61] loss: 0.01109 time model: 0.00652 acc: 0.88854
80
[epoch: 1, batch:     81] loss: 0.01108 time model: 0.00651 acc: 0.89023


In [35]:
LOGFILE_PATH = 'logs/eval/full_dataset_fgsm20'
trainer_full_dataset_fgsm20 = BaseTrainer(model=full_dataset_fgsm20, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='eval_normalTrain1', 
                                   n_epochs=1)

trainer_full_dataset_fgsm20.model.eval()
trainer_full_dataset_fgsm20 = trainer_full_dataset_fgsm20.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=None)

20
[epoch: 2, batch:     21] loss: 0.04389 time model: 0.00661 acc: 0.56406
40
[epoch: 2, batch:     41] loss: 0.04621 time model: 0.00654 acc: 0.54844
60
[epoch: 2, batch:     61] loss: 0.04587 time model: 0.00652 acc: 0.55573
80
[epoch: 2, batch:     81] loss: 0.04547 time model: 0.00650 acc: 0.55977


In [36]:
LOGFILE_PATH = 'logs/eval/full_dataset_pdg030'
trainer_full_dataset_pdg03 = BaseTrainer(model=full_dataset_pdg03, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='eval_normalTrain1', 
                                   n_epochs=1)

trainer_full_dataset_pdg03.model.eval()
trainer_full_full_dataset_pdg03 = trainer_full_dataset_pdg03.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=None)

20
[epoch: 1, batch:     21] loss: 0.01020 time model: 0.00665 acc: 0.88906
40
[epoch: 1, batch:     41] loss: 0.00962 time model: 0.00656 acc: 0.89531
60
[epoch: 1, batch:     61] loss: 0.00995 time model: 0.00653 acc: 0.89740
80
[epoch: 1, batch:     81] loss: 0.00980 time model: 0.00651 acc: 0.89883


### Attack: FGSM

In [41]:
LOGFILE_PATH = 'logs/eval/full_dataset_normal_fgsm'
trainer_full_dataset_normal = BaseTrainer(model=full_dataset_normal, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='eval_full_dataset_normal_fgsm', 
                                   n_epochs=1)

trainer_full_dataset_normal.model.eval()
trainer_full_dataset_normal = trainer_full_dataset_normal.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=fgsm)

20
[epoch: 1, batch:     21] loss: 0.28684 time model: 0.02669 acc: 0.02500
40
[epoch: 1, batch:     41] loss: 0.29466 time model: 0.02658 acc: 0.02656
60
[epoch: 1, batch:     61] loss: 0.29380 time model: 0.02656 acc: 0.02708
80
[epoch: 1, batch:     81] loss: 0.29359 time model: 0.02655 acc: 0.02383


In [44]:
LOGFILE_PATH = 'logs/eval/full_dataset_normal_pgd'
trainer_full_dataset_normal = BaseTrainer(model=full_dataset_normal, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='eval_full_dataset_normal_fgsm', 
                                   n_epochs=1)

trainer_full_dataset_normal.model.eval()
trainer_full_dataset_normal = trainer_full_dataset_normal.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=pgd_linf)

20
[epoch: 1, batch:     21] loss: 0.25050 time model: 0.14552 acc: 0.03906
40
[epoch: 1, batch:     41] loss: 0.24649 time model: 0.14537 acc: 0.04922
60
[epoch: 1, batch:     61] loss: 0.24717 time model: 0.14549 acc: 0.05260
80
[epoch: 1, batch:     81] loss: 0.25027 time model: 0.14559 acc: 0.05273


In [45]:
LOGFILE_PATH = 'logs/eval/full_dataset_fgsm03_fgsm_a'
trainer_full_dataset_fgsm03 = BaseTrainer(model=full_dataset_fgsm03, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='eval_full_dataset_fgsm03_fgsm', 
                                   n_epochs=1)

trainer_full_dataset_fgsm03.model.eval()
trainer_full_dataset_fgsm03 = trainer_full_dataset_fgsm03.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=fgsm)

20
[epoch: 1, batch:     21] loss: 0.00736 time model: 0.02667 acc: 0.92500
40
[epoch: 1, batch:     41] loss: 0.00708 time model: 0.02657 acc: 0.93359
60
[epoch: 1, batch:     61] loss: 0.00720 time model: 0.02655 acc: 0.93281
80
[epoch: 1, batch:     81] loss: 0.00730 time model: 0.02653 acc: 0.93359


In [46]:
LOGFILE_PATH = 'logs/eval/full_dataset_fgsm20_fgsm_a'
trainer_full_dataset_fgsm20 = BaseTrainer(model=full_dataset_fgsm20, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='eval_full_dataset_fgsm20_fgsm', 
                                   n_epochs=1)

trainer_full_dataset_fgsm20.model.eval()
trainer_full_dataset_fgsm20 = trainer_full_dataset_fgsm20.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=fgsm)

20
[epoch: 1, batch:     21] loss: 0.21727 time model: 0.02634 acc: 0.02344
40
[epoch: 1, batch:     41] loss: 0.21897 time model: 0.02637 acc: 0.01875
60
[epoch: 1, batch:     61] loss: 0.21923 time model: 0.02638 acc: 0.01458
80
[epoch: 1, batch:     81] loss: 0.21898 time model: 0.02639 acc: 0.01328


In [22]:
LOGFILE_PATH = 'logs/eval/full_dataset_pdg030_fgsm'
trainer_full_dataset_pdg03 = BaseTrainer(model=full_dataset_pdg03, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='full_dataset_pdg030_fgsm', 
                                   n_epochs=1)

trainer_full_dataset_pdg03.model.eval()
trainer_full_full_dataset_pdg03 = trainer_full_dataset_pdg03.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=fgsm)

20
[epoch: 1, batch:     21] loss: 0.11575 time model: 0.02652 acc: 0.12656
40
[epoch: 1, batch:     41] loss: 0.11749 time model: 0.02652 acc: 0.10859
60
[epoch: 1, batch:     61] loss: 0.12037 time model: 0.02654 acc: 0.09688
80
[epoch: 1, batch:     81] loss: 0.12010 time model: 0.02655 acc: 0.09375


### Attack PDG_03


In [48]:
global_eps = 0.3

def fgsm(model, X, y, epsilon=0.3):
    """ Construct FGSM adversarial examples on the examples X"""
    epsilon = global_eps
    delta = torch.zeros_like(X, requires_grad=True)
    delta = delta.cuda()
    loss = nn.CrossEntropyLoss()(model(X + delta), y)
    loss.backward()
    return epsilon * delta.grad.detach().sign()


def pgd_linf(model, X, y, epsilon=0.1, alpha=0.01, num_iter=7, randomize=False):
    """ Construct PGD adversarial examples on the examples X"""
    epsilon = global_eps
    if randomize:
        delta = torch.rand_like(X, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon
    else:
        delta = torch.zeros_like(X, requires_grad=True)
        
    for t in range(num_iter):
        loss = nn.CrossEntropyLoss()(model(X + delta), y)
        loss.backward()
        delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon)
        delta.grad.zero_()
    return delta.detach()

In [50]:
LOGFILE_PATH = 'logs/eval/full_dataset_normal_pgd'
trainer_full_dataset_normal = BaseTrainer(model=full_dataset_normal, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='eval_full_dataset_normal_pdg', 
                                   n_epochs=1)

trainer_full_dataset_normal.model.eval()
trainer_full_dataset_normal = trainer_full_dataset_normal.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=pgd_linf)

Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/anaconda3/lib/python3.7/

KeyboardInterrupt: 

In [20]:
LOGFILE_PATH = 'logs/eval/full_dataset_fgsm03_pgd_a'
trainer_full_dataset_fgsm03 = BaseTrainer(model=full_dataset_fgsm03, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='eval_full_dataset_fgsm03_pgd', 
                                   n_epochs=1)

trainer_full_dataset_fgsm03.model.eval()
trainer_full_dataset_fgsm03 = trainer_full_dataset_fgsm03.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=pgd_linf)

20
[epoch: 1, batch:     21] loss: 0.35759 time model: 0.15166 acc: 0.02656
40
[epoch: 1, batch:     41] loss: 0.35994 time model: 0.14841 acc: 0.01875
60
[epoch: 1, batch:     61] loss: 0.36105 time model: 0.14732 acc: 0.01771
80
[epoch: 1, batch:     81] loss: 0.36105 time model: 0.14679 acc: 0.01836


In [21]:
LOGFILE_PATH = 'logs/eval/full_dataset_fgsm20_pgd_a'
trainer_full_dataset_fgsm20 = BaseTrainer(model=full_dataset_fgsm20, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='eval_full_dataset_fgsm20_pgd', 
                                   n_epochs=1)

trainer_full_dataset_fgsm20.model.eval()
trainer_full_dataset_fgsm20 = trainer_full_dataset_fgsm20.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=pgd_linf)

20
[epoch: 1, batch:     21] loss: 0.18556 time model: 0.14512 acc: 0.02969
40
[epoch: 1, batch:     41] loss: 0.18164 time model: 0.14515 acc: 0.03672
80
[epoch: 1, batch:     81] loss: 0.18309 time model: 0.14530 acc: 0.03516


In [23]:
LOGFILE_PATH = 'logs/eval/full_dataset_pdg030_pgd'
trainer_full_dataset_pdg03 = BaseTrainer(model=full_dataset_pdg03, 
                                   train_dl=train_data_loader, 
                                   valid_dl=valid_data_loader, 
                                   test_dl=valid_data_loader, 
                                   criterion=criterion, 
                                   model_filename='full_dataset_pdg030_pgd', 
                                   n_epochs=1)

trainer_full_dataset_pdg03.model.eval()
trainer_full_full_dataset_pdg03 = trainer_full_dataset_pdg03.run_epoch(0, valid_data_loader, 
                                                      optimizer=None, 
                                                      attack=pgd_linf)

20
[epoch: 1, batch:     21] loss: 0.02575 time model: 0.14571 acc: 0.70312
40
[epoch: 1, batch:     41] loss: 0.02659 time model: 0.14564 acc: 0.68594
60
[epoch: 1, batch:     61] loss: 0.02633 time model: 0.14561 acc: 0.68802
80
[epoch: 1, batch:     81] loss: 0.02566 time model: 0.14552 acc: 0.69688
